# Hugging-face transformers trained on mathlib

## Initialize

In [ ]:
#INIT (If colab)
!git clone https://github.com/nick-kuhn/leantools.git

In [ ]:
!pip install transformers==4.19.0
!pip install datasets==1.16.0
!pip install huggingface-hub==0.1.0
!pip install arguments
!pip install consoleprinter
!pip install accelerate 
!pip install wandb==0.12.0
!pip install tensorboard==2.6.0
#!pip install torch==1.11.0
!sudo apt-get install git-lfs
%cd leantools
!git lfs install 
!git config user.name "***"
!git config user.email "***@***.**"
%cd ..

In [ ]:
!git config --global credential.helper store
!wandb login 
!accelerate config
!huggingface-cli login

# Training and updating the model

## Run the main script

In [ ]:
!accelerate launch ./leantools/parrot/train_parrot.py \
--model_ckpt ntkuhn/lean-parrot \
--train_batch_size 12 \
--valid_batch_size 12 \
--learning_rate 5e-4 \
--num_warmup_steps 2000 \
--gradient_accumulation 1 \
--gradient_checkpointing True \
--max_train_steps 150000 \
--save_checkpoint_steps 5000 \
--resume_from_checkpoint "./lean-parrot/step_20000"

### Update main branch to latest model

In [ ]:
!accelerate launch ./leantools/parrot/update_from_checkpoint.py \
--model_ckpt ntkuhn/lean-parrot \
--train_batch_size 12 \
--valid_batch_size 12 \
--learning_rate 5e-4 \
--num_warmup_steps 2000 \
--gradient_accumulation 1 \
--gradient_checkpointing True \
--max_train_steps 150000 \
--save_checkpoint_steps 5000 \
--resume_from_checkpoint "./lean-parrot/step_35000"

## Push to GIT


In [ ]:
%cd leantools
!git add parrot/train_parrot.py parrot/arguments.py parrot/update_from_checkpoint.py
!git commit -m "Commit message"
!git push
%cd ..

# Prepare data and tokenizer (only do once)

## Convert code files to a json database

---



In [ ]:
import os
from google.colab import files

#for root, dirs, files in os.walk('/tmp'):
#    if os.path.basename(root) != 'modules':
#        continue
#    data = [parse_file(os.path.join(root,f)) for f in files]

#Here, manually load src (i.e. mathlib) into leantools

mathlib_dict = {}
for root, dirs, files in os.walk('leantools/src'):
  for f in files:
    with open(os.path.join(root,f)) as file:
      mathlib_dict.update({os.path.join(root,f) : file.read()})


In [ ]:
#Write everything in json file:
import json

with open('mathlib.json', 'w') as json_file:
  json.dump(mathlib_dict, json_file)


!dir

In [ ]:
!git a -- 'mathlib.json' 

lean_assemble.py  out.lean  README.md  tokenizer.py  token.list


## Read in the database and train the tokenizer

In [ ]:
from datasets import load_dataset
mathlib = load_dataset("json", data_files = "leantools/mathlib.jsonl", split = "train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-015c5b120cd8b6e0/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


In [ ]:
mathlib['data'][1]

"/-\nCopyright (c) 2022 Andrew Yang. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Andrew Yang\n-/\nimport algebraic_geometry.gluing\nimport category_theory.limits.opposites\nimport algebraic_geometry.Gamma_Spec_adjunction\n\n/-!\n# Fibred products of schemes\n\nIn this file we construct the fibred product of schemes via gluing.\nWe roughly follow [har77] Theorem 3.3.\n\nIn particular, the main construction is to show that for an open cover `{ Uᵢ }` of `X`, if there\nexist fibred products `Uᵢ ×[Z] Y` for each `i`, then there exists a fibred product `X ×[Z] Y`.\n\nThen, for constructing the fibred product for arbitrary schemes `X, Y, Z`, we can use the\nconstruction to reduce to the case where `X, Y, Z` are all affine, where fibred products are\nconstructed via tensor products.\n\n-/\nuniverses v u\nnoncomputable theory\n\nopen category_theory category_theory.limits algebraic_geometry\nnamespace algebraic_geometry.Scheme\n\nnamespace

## Train the Tokenizer

In [ ]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode
from transformers import AutoTokenizer, HfArgumentParser


tokenizer = AutoTokenizer.from_pretrained("gpt2")

base_vocab = list(bytes_to_unicode().values()) #Probably not desirable: We want whitespaces to persist for formatting?!

In [ ]:
#Convert from unicode bytes to int, then back
char = 'ᵢ'
bytes.fromhex(hex(int(char.encode(encoding = 'utf8').hex(), 16)+6)[2:]).decode("utf8")

'ᵨ'

In [ ]:
new_char_list = []
for data_point in mathlib["data"]:
  for char in data_point:
    if char not in base_vocab and char not in new_char_list:
      new_char_list.append(char)

In [ ]:
#The result is this:
new_char_list = ['\n', ' ', '⟶', '⟨', '⟩', '←', '∈', '≫', '⊤', 'ᵢ', '𝒰', '∀',
    'ⱼ', '𝟙', 'ₖ', 'λ', '⁻', '∩', 'π', 'ι', '≪', '≅', '₁', '₂', 'ᵒ', 'ᵖ', 'α', 
    'β', '→', '⥤', '⋙', 'Γ', '↑', '⤳', '₃', '₄', '₆', 'ˣ', 'ℚ', '⊣', '𝟭', 
    '∉', '↔', '𝒪', '▸', '↓', '∃', '⊓', '≤', '⊆', '∐', '⇉', '𝖣', 'Σ', '∨', 
    '∧', '∅', 'Π', '⨆', '𝔭', '𝔮', '⊢', '⋃', 'ℕ', '∑', 'ₓ', '≃', 'ₜ', '∘', '≠',
    '⇑', '⊥', 'ᶜ', '⊔', '≌', '꙳', '⋆', '⊕', '⨅', 'ᵈ', '∪', '⋂', '⁺', '↥', 
    '𝒥', '𝒜', 'γ', '↦', '⁰', '₀', 'ₙ', 'δ', '–', '⦃', '⦄', 'ℤ', 'κ', 'φ', 'ε', 
    '₅', '‹', '›', 'σ', '∣', '⇒', '↪', '↿', 'ˢ', '𝒫', '•', 'ₗ', 'ₐ', 'τ', 'ζ', 
    '“', '”', '⟦', '⟧', '≈', 'η', '≡', '⧸', 'ϕ', '⁅', '⁆', 'ᵐ', '⅟', '⟮', '⟯', 
    'ᵀ', '⬝', 'ₛ', 'ₘ', '∏', '⋯', '∙', 'ℝ', 'ᵤ', '≥', '⊗', 'ξ', 'μ', 'ϖ', 'ᶠ',
    'ᵣ', '⊂', 'ₚ', '⌟', 'ℐ', '⨁', '𝕎', '…', 'Φ', 'ˡ', 'ⁱ', 'ψ', 'χ', '̂', 'ⁿ',
    '𝓟', 'ν', 'ℂ', '∥', '₊', 'ʸ', 'ℍ', 'ℵ', '≺', 'ᵃ', 'ᵥ', '⅓', '◃', '∆', '₇', 
    '₈', '𝕜', '⊞', 'ℬ', '∞', '√', '†', 'ᘁ', 'š', '⌊', '⌋', '⌈', '⌉', 'ℳ', 'ρ', 
    '↾', '↟', 'ᵇ', '₋', 'ₒ', '≰', '⨂', 'ω', 'θ', 'ᴴ', '⟪', '⟫', 'ℙ', '⨯', '◫', 
    '↠', '↻', '⋊', '𝒢', 'ℋ', '∫', '𝒟', '𝒮', '◁', '▷', 'Δ', '𝒞', '⨿', '↗', 
    '↘', 'ℰ', '⟹', '𝒯', 'ᴹ', '⊚', 'ℱ', '─', '│', '𝓝', '\xa0', '⊇', '𝓤', '○', 
    '∋', '‘', '🎉', '𝓣', '𝓑', '𝕝', '⋖', 'ℓ', '̀', 'Ψ', 'ₕ', '∼', 'ő', '𝔖', '𝓢', 
    '′', '𝔠', '≼', '♯', '⨳', '⧏', 'Ω', 'Λ', '𝓞', '𝔽', '∤', '∂', '𝔼', 'ℒ', 'Ι', 
    'ᵏ', '∯', '∮', 'Θ', '⨍', '𝓕', '⋁', '💥', '📋', '⊹', '├', '┌', '⇐', '✅', 
    '❌', '🛑', '⊑', '≻', '𝕂', '𝔸', 'ᵗ', 'ʰ', '𝔹', 'ᗮ', '™', '‵', '𝕆', '⩿', 
    '𝒄', '⇔', '⁴', 'υ', '\u200c', '𝓚', '⋮', '‖', '⊙', '⋈', '⊛', '𝓒', '∠', '’', 
    '𝓘', '𝓡', '𝒅', '𝑳', '𝑹', 'ᴸ', '⊨', 'ₑ', 'ś', 'ᾰ']


['\n', ' ', '⟶', '⟨', '⟩', '←', '∈', '≫', '⊤', 'ᵢ', '𝒰', '∀', 'ⱼ', '𝟙', 'ₖ', 'λ', '⁻', '∩', 'π', 'ι', '≪', '≅', '₁', '₂', 'ᵒ', 'ᵖ', 'α', 'β', '→', '⥤', '⋙', 'Γ', '↑', '⤳', '₃', '₄', '₆', 'ˣ', 'ℚ', '⊣', '𝟭', '∉', '↔', '𝒪', '▸', '↓', '∃', '⊓', '≤', '⊆', '∐', '⇉', '𝖣', 'Σ', '∨', '∧', '∅', 'Π', '⨆', '𝔭', '𝔮', '⊢', '⋃', 'ℕ', '∑', 'ₓ', '≃', 'ₜ', '∘', '≠', '⇑', '⊥', 'ᶜ', '⊔', '≌', '꙳', '⋆', '⊕', '⨅', 'ᵈ', '∪', '⋂', '⁺', '↥', '𝒥', '𝒜', 'γ', '↦', '⁰', '₀', 'ₙ', 'δ', '–', '⦃', '⦄', 'ℤ', 'κ', 'φ', 'ε', '₅', '‹', '›', 'σ', '∣', '⇒', '↪', '↿', 'ˢ', '𝒫', '•', 'ₗ', 'ₐ', 'τ', 'ζ', '“', '”', '⟦', '⟧', '≈', 'η', '≡', '⧸', 'ϕ', '⁅', '⁆', 'ᵐ', '⅟', '⟮', '⟯', 'ᵀ', '⬝', 'ₛ', 'ₘ', '∏', '⋯', '∙', 'ℝ', 'ᵤ', '≥', '⊗', 'ξ', 'μ', 'ϖ', 'ᶠ', 'ᵣ', '⊂', 'ₚ', '⌟', 'ℐ', '⨁', '𝕎', '…', 'Φ', 'ˡ', 'ⁱ', 'ψ', 'χ', '̂', 'ⁿ', '𝓟', 'ν', 'ℂ', '∥', '₊', 'ʸ', 'ℍ', 'ℵ', '≺', 'ᵃ', 'ᵥ', '⅓', '◃', '∆', '₇', '₈', '𝕜', '⊞', 'ℬ', '∞', '√', '†', 'ᘁ', 'š', '⌊', '⌋', '⌈', '⌉', 'ℳ', 'ρ', '↾', '↟', 'ᵇ', '₋', 'ₒ', '≰', '⨂', 'ω', 'θ', 'ᴴ', '⟪'

In [ ]:
dataset = mathlib
iter_dataset = iter(dataset['data'])

In [ ]:
new_tokenizer = tokenizer.train_new_from_iterator(iter_dataset, vocab_size = 200_000, initial_alphabet = base_vocab + new_char_list[2:])

In [ ]:
#Aside:
#Here is how it works:
from transformers import GPT2Tokenizer

word = new_tokenizer.get_vocab()
new_tokenizer.tokenize('ᵢ') # --> 'áµ¢'
print([hex(ord('á')), hex(ord('µ')), hex(ord('¢'))],
  'ᵢ'.encode())

['0xe1', '0xb5', '0xa2'] b'\xe1\xb5\xa2'


In [ ]:
#Generate and display decoded token list
token_list = []
for token in new_tokenizer.get_vocab().keys():
  token_list.append(new_tokenizer.convert_tokens_to_string(token))
print(token_list) 

[' Thanks', ' h𝒜', 'ℓp', 'dependently', ' Street', ' uniformity', 'hlifts', ' coinitial', ' load', 'tropicalization', ' Blowing', ' _⟩', ' Infer', ' \\}$', ' Apply', ' Teichmuller', ' hCFk', 'profinite', ' Trailing', ' prim', 'ushout', 'insert', ' demo', ' ζ', 'hbI', ' sma', ' ((⋃', ' skeletons', 'tauto', ' listed', ' hItop', ' Hpq', 'roots', 'arctan', ' rescale', 'Nested', 'roid', ' Sequences', 'laine', ' Math', 'Odd', ' freiman', 'ops', 'Inject', 'Lens', 'pcontinuous', ' bijections', ' nk', 'iseqv', 'ulower', ' interp', ']})', ' Render', 'ørm', 'params', ' enlargements', ' RS', 'Taylor', 'Problem', 'enike', 'ermain', 'purely', ' infix', ' aff', ' Backwards', 'Case', ' ne', 'mmap', 'Pa', 'npow', 'ceux', 'rapidly', ' sna', ' qp', ' unreduced', ' ⋊[`:', 'mich', ')]', 'Hazewink', 'ther', 'ν', ' asymptotics', '\n                                         ', ' heqv', ' stabilizer', 'lobular', 'define', ' Zhang', 'groupprops', ' IΨ', ' multi', ' assignment', ' Semin', ']]`.', 'containined', '

In [ ]:
#Save the tokenizer
new_tokenizer.save_pretrained("./leantools/parrot/tokenizer/")

('./leantools/parrot/tokenizer/tokenizer_config.json',
 './leantools/parrot/tokenizer/special_tokens_map.json',
 './leantools/parrot/tokenizer/vocab.json',
 './leantools/parrot/tokenizer/merges.txt',
 './leantools/parrot/tokenizer/added_tokens.json',
 './leantools/parrot/tokenizer/tokenizer.json')

# Testing different parts of the Script

## Initialize the Transformer

In [ ]:
#Tokenizer
#tokenizer = new_tokenizer #if trained
#from saved:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./leantools/parrot/tokenizer/")

# Configuration
config_kwargs = {"vocab_size": len(tokenizer),
                 "scale_attn_by_layer_idx": True,
                 "reorder_and_upcast_attn": True}

In [ ]:

#from arguments import InitializationArguments
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, HfArgumentParser


# Load model with config and push to hub
config = AutoConfig.from_pretrained( "gpt2-large", **config_kwargs)
model = AutoModelForCausalLM.from_config(config)
#model.save_pretrained(args.model_name, push_to_hub=args.push_to_hub)

## Training Loop

In [ ]:
#!pip install argparse
from argparse import Namespace
from accelerate import Accelerator, DistributedType
from transformers import set_seed
import wandb
%env "WANDB_NOTEBOOK_NAME" "Parrot"
wandb.login() # Somehow doesn't appear as tracker?! 

#Settings
parser = HfArgumentParser(TrainingArguments)
args = parser.parse_known_args()

#Accelerator
accelerator = Accelerator(log_with=["wandb", "tensorboard"], logging_dir=f"{args[0].save_dir}/log")
acc_state = {str(k): str(v) for k, v in accelerator.state.__dict__.items()}
args = Namespace(**vars(args[0]), **acc_state)

samples_per_step = accelerator.state.num_processes * args.train_batch_size
set_seed(args.seed)

env: "WANDB_NOTEBOOK_NAME"="Parrot"


In [ ]:
from huggingface_hub import Repository

# Clone model repository
if accelerator.is_main_process:
    hf_repo = Repository(args.save_dir, clone_from=args.model_ckpt)
    hf_repo.git_pull()
#help(Repository)

Cloning https://huggingface.co/codeparrot/codeparrot into local empty directory.


Download file pytorch_model.bin:   0%|          | 4.18k/5.75G [00:00<?, ?B/s]

Download file wandb/run-20220114_155039-1uxxfypf/logs/debug-internal.log:   0%|          | 15.3k/256M [00:00<?…

Download file runs/Nov11_21-44-48_leandro-16x-v100/events.out.tfevents.1636667088.leandro-16x-v100.23074.0:   …

Download file log/debug_0.log:   0%|          | 3.47k/190M [00:00<?, ?B/s]

Download file wandb/run-20220114_155039-1uxxfypf/run-1uxxfypf.wandb:   0%|          | 2.79k/302M [00:00<?, ?B/…

Download file wandb/run-20211111_214446-3xgbgfj2/logs/debug-internal.log:   0%|          | 3.12k/411M [00:00<?…

Download file wandb/run-20211111_214446-3xgbgfj2/run-3xgbgfj2.wandb:   0%|          | 16.0k/280M [00:00<?, ?B/…

Download file wandb/run-20220114_155039-1uxxfypf/files/output.log:   0%|          | 16.0k/182M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
import logging
from pathlib import Path
import datasets 
import transformers 

def setup_logging(args):
    project_name = args.model_ckpt.split("/")[-1]
    logger = logging.getLogger(__name__)
    log_dir = Path(args.save_dir) / "log/"
    log_dir.mkdir(exist_ok=True)
    filename = f"debug_{accelerator.process_index}.log"
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
        handlers=[logging.FileHandler(log_dir / filename), logging.StreamHandler()],
    )
    if accelerator.is_main_process:  # we only want to setup logging once
        accelerator.init_trackers(project_name, vars(args))
        run_name = accelerator.trackers[0].run.name
        logger.setLevel(logging.INFO)
        datasets.utils.logging.set_verbosity_info()
        transformers.utils.logging.set_verbosity_info()
    else:
        run_name = ""
        logger.setLevel(logging.ERROR)
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
    return logger, run_name


In [ ]:
help(accelerator.init_trackers)

Help on method init_trackers in module accelerate.accelerator:

init_trackers(project_name: str, config: Union[dict, NoneType] = None, init_kwargs: Union[dict, NoneType] = {}) method of accelerate.accelerator.Accelerator instance
    Initializes a run for all trackers stored in `self.log_with`, potentially with starting configurations
    
    Args:
        project_name (`str`):
            The name of the project. All trackers will save their data based on this
        config (`dict`, *optional*):
            Optional starting configuration to be logged.
        init_kwargs (`dict`, *optional*):
            A nested dictionary of kwargs to be passed to a specific tracker's `__init__` function. Should be
            formatted like this:
            ```python
            {"wandb": {"tags": ["tag_a", "tag_b"]}}
            ```



In [ ]:

# Logging (Doesn't work in colab)
logger, run_name = setup_logging(args)
logger.info(accelerator.state)

AttributeError: ignored

In [ ]:
run_name = "run_one"
# Checkout new branch on repo
if accelerator.is_main_process:
    hf_repo.git_checkout(run_name, create_branch_ok=True)

Revision `run_one` does not exist. Created and checked out branch `run_one`.



In [ ]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(args.save_dir)
if args.gradient_checkpointing:
    model.gradient_checkpointing_enable()
tokenizer = AutoTokenizer.from_pretrained(args.save_dir)

## Testing the data loader

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import IterableDataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.datapipes.iter.combinatorics import ShufflerIterDataPipe
class ConstantLengthDataset(IterableDataset):
    """
    Iterable dataset that returns constant length chunks of tokens from stream of text files.
        Args:
            tokenizer (Tokenizer): The processor used for proccessing the data.
            dataset (dataset.Dataset): Dataset with text files.
            infinite (bool): If True the iterator is reset after dataset reaches end else stops.
            seq_length (int): Length of token sequences to return.
            num_of_sequences (int): Number of token sequences to keep in buffer.
            chars_per_token (int): Number of characters per token used to estimate number of tokens in text buffer.
            tokenized (bool): If true we use a pretokenized dataset.
    """

    def __init__(
        self,
        tokenizer,
        dataset,
        infinite=False,
        seq_length=1024,
        num_of_sequences=1024,
        chars_per_token=3.6,
        tokenized=False,
    ):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.bos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.epoch = 0
        self.infinite = infinite
        self.current_size = 0
        self.tokenized = tokenized

        if self.tokenized:
            self.max_buffer_size = seq_length * num_of_sequences
            self.content_field = "input_ids"
        else:
            self.max_buffer_size = seq_length * chars_per_token * num_of_sequences
            self.content_field = "content"

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.max_buffer_size:
                    break
                try:
                    buffer.append(next(iterator)[self.content_field])
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    if self.infinite:
                        iterator = iter(self.dataset)
                        self.epoch += 1
                        logger.info(f"Dataset epoch: {self.epoch}")
                    else:
                        more_examples = False
                        break
            if self.tokenized:
                tokenized_inputs = buffer
            else:
                tokenized_inputs = self.tokenizer(buffer, truncation=False)["input_ids"]
            all_token_ids = []
            for tokenized_input in tokenized_inputs:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])
            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    self.current_size += 1
                    yield torch.tensor(input_ids)

    def shuffle(self, buffer_size=1000):
        return ShufflerIterDataPipe(self, buffer_size=buffer_size)

In [ ]:
def create_dataloaders(args): #adapted for our purposes
    # Could improve this by first creating distinct training and validation sets
    ds_kwargs = {"streaming": True}
    train_data = load_dataset('json', data_files = "./leantools/mathlib.jsonl", streaming = True, split = "train[:80%]")
    train_data = train_data.shuffle(buffer_size=args.shuffle_buffer, seed=args.seed)
    valid_data = load_dataset('json', data_files = "./leantools/mathlib.jsonl", streaming = True, split = "train[80%:]")

    train_dataset = ConstantLengthDataset(tokenizer, train_data, infinite=True, seq_length=args.seq_length)
    valid_dataset = ConstantLengthDataset(tokenizer, valid_data, infinite=False, seq_length=args.seq_length)

    train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size)
    eval_dataloader = DataLoader(valid_dataset, batch_size=args.valid_batch_size)
    return train_dataloader, eval_dataloader

In [ ]:
import datasets
mathlib_data = load_dataset('json', data_files = "./leantools/mathlib.jsonl",  split = "train")
mathlib_data_train_val_test = mathlib_data.train_test_split(train_size=0.8, seed=42)

In [ ]:
mathlib_data_val_test = mathlib_data_train_val_test.pop("test").train_test_split(train_size = 0.5, seed = 42)
mathlib_data_train_val_test["validation"] = mathlib_data_val_test["train"]
mathlib_data_train_val_test["test"] = mathlib_data_val_test["test"]

In [ ]:
for split, dataset in mathlib_data_train_val_test.items():
    dataset.to_json(f"leantools/mathlib-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_data = load_dataset('json', data_files = "./leantools/mathlib.jsonl", streaming = True, split = "train[:80%]")
valid_data = load_dataset('json', data_files = "./leantools/mathlib.jsonl", streaming = True, split = "train[80%:]")

ValueError: ignored

In [ ]:
list(train_data.take(1))

[{'name': 'leantools/src/algebraic_geometry/function_field.lean',
  'data': "/-\nCopyright (c) 2022 Andrew Yang. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Andrew Yang\n-/\nimport algebraic_geometry.properties\n\n/-!\n# Function field of integral schemes\n\nWe define the function field of an irreducible scheme as the stalk of the generic point.\nThis is a field when the scheme is integral.\n\n## Main definition\n* `algebraic_geometry.Scheme.function_field`: The function field of an integral scheme.\n* `algebraic_geometry.germ_to_function_field`: The canonical map from a component into the function\n  field. This map is injective.\n-/\n\nuniverses u v\n\nopen topological_space opposite category_theory category_theory.limits Top\n\nnamespace algebraic_geometry\n\nvariable (X : Scheme)\n\n/-- The function field of an irreducible scheme is the local ring at its generic point.\nDespite the name, this is a field only when the scheme is

## Main Training Loop

In [ ]:
help(model.train)

Help on method train in module torch.nn.modules.module:

train(mode: bool = True) -> ~T method of transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel instance
    Sets the module in training mode.
    
    This has any effect only on certain modules. See documentations of
    particular modules for details of their behaviors in training/evaluation
    mode, if they are affected, e.g. :class:`Dropout`, :class:`BatchNorm`,
    etc.
    
    Args:
        mode (bool): whether to set training mode (``True``) or evaluation
                     mode (``False``). Default: ``True``.
    
    Returns:
        Module: self



## Example data

In [ ]:
print(next(iter_dataset))

/-
Copyright (c) 2022 Andrew Yang. All rights reserved.
Released under Apache 2.0 license as described in the file LICENSE.
Authors: Andrew Yang
-/
import algebraic_geometry.gluing
import category_theory.limits.opposites
import algebraic_geometry.Gamma_Spec_adjunction

/-!
# Fibred products of schemes

In this file we construct the fibred product of schemes via gluing.
We roughly follow [har77] Theorem 3.3.

In particular, the main construction is to show that for an open cover `{ Uᵢ }` of `X`, if there
exist fibred products `Uᵢ ×[Z] Y` for each `i`, then there exists a fibred product `X ×[Z] Y`.

Then, for constructing the fibred product for arbitrary schemes `X, Y, Z`, we can use the
construction to reduce to the case where `X, Y, Z` are all affine, where fibred products are
constructed via tensor products.

-/
universes v u
noncomputable theory

open category_theory category_theory.limits algebraic_geometry
namespace algebraic_geometry.Scheme

namespace pullback

variables {C : Type

## Initialize the Transformer